In [ ]:

### This notebook is used to convert ROI on single slices of the tiff-stacks into contigous detections.

In [1]:
%pylab inline
import numpy as np
import sys
sys.path.append('../src')
import det_sort as ds
from convert_ROI_to_cell_volumes import ConvertROItoCellVolumes
from ijroi.ij_roi import Roi
from ijroi.ijpython_encoder import encode_ij_roi,RGB_encoder
from ijroi.ijpython_decoder import decode_ij_roi

import tifffile


Populating the interactive namespace from numpy and matplotlib


In [2]:
###Parameters.

img_width = 512 #pixels
img_height = 512 #pixels
border_offset = 12 #If regions end or start within this they are included.
scale = 0.26 #um/pixel
zspacing = 0.50 #physical distance (um) between z-slices.
filepath = "/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/acquisitions/slide3/"
outpath  =  "/Users/dwaithe/Desktop/out03/"

convObj = ConvertROItoCellVolumes(img_width, img_height, border_offset, scale, zspacing, filepath, outpath)
convObj.append_new_regions(outpath, extend_roi=False)
                      

line 35137 excluded as was not complete.


In [ ]:
convObj.plot_reg(outpath+"/img_stk_x_-5000.6y_5008.4.tif")

In [ ]:
cell_data = []

    
ref_loc = convObj.ref_loc
store_cell_data = []
store_file_names = []
for pathname in ref_loc:
            coords = pathname.split("_")
            pathname2 ='img_stk_x_'+coords[0]+"y_"+coords[1]+".tif"
            store_file_names.append(pathname2)
            input_file = filepath+pathname2
            input_test = "/Users/dwaithe/Desktop/out03/"+pathname2
            tfile = tifffile.TiffFile(input_test)
            img_stk = tfile.asarray()
            slices = ref_loc[pathname] 
            data = []
           
            roi_array = convObj.return_overlay(tfile)


            #plot_reg(img_stk, roi_array)
            data = convObj.return_data_from_reg(img_stk, roi_array,'sum')
            for cell in data:

                store_cell_data.append(data[cell])
            tfile.close()
cell_data.append(np.copy(store_cell_data))

In [ ]:
import pyperclip
stg = ""
for txt in store_file_names:
    stg += str(txt) +'\n'
pyperclip.copy(stg)
spam = pyperclip.paste()

In [ ]:
plot(cell_data[0])

In [ ]:
bins = np.linspace(0,16000,100)
cell_total = list(meant[0])
a,b = np.histogram(cell_total,bins=bins)
plot(b[1:],a)
cell_total = list(meant[1])
a,b = np.histogram(cell_total,bins=bins)
plot(b[1:],a)
cell_total = list(meant[2])
a,b = np.histogram(cell_total,bins=bins)
plot(b[1:],a)


In [ ]:
bins = np.linspace(0,180000,20)
cell_total = list(volume[0])
a,b = np.histogram(cell_total,bins=bins)
plot(b[1:],a)
cell_total = list(volume[1])
a,b = np.histogram(cell_total,bins=bins)
plot(b[1:],a)
cell_total = list(volume[2])
a,b = np.histogram(cell_total,bins=bins)
plot(b[1:],a)




In [ ]:
bins = np.linspace(0,12000,20)
cell_total = list(area[0])
a,b = np.histogram(cell_total,bins=bins)
plot(b[1:],a)
cell_total = list(area[1])
a,b = np.histogram(cell_total,bins=bins)
plot(b[1:],a)
cell_total = list(area[2])
a,b = np.histogram(cell_total,bins=bins)
plot(b[1:],a)



In [ ]:
for i in range(0,3):
    plot(sumt[i],volume[i],'o')

In [ ]:
import tifffile
import numpy as np

pathfile = "/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/results/"
f=open(pathfile+"file_creation_list.txt", "r")
fl =f.readlines()
creation_array = []
file_name_array = []
number_of_slices = []
for x in fl:
	if x.split(" ").__len__() >1:
		file_name = x.split(" ")[0]
		timet = x.split(" ")[-2].split(":")
		if timet.__len__() == 3:
			
			
			try:
				input_file = "/Users/dwaithe/Documents/collaborators/WaitheD/micro_vision/acquisitions/slide3/data_correct_dim_order/"
				tfile = tifffile.TiffFile(input_file+file_name)
				creation_array.append(float(timet[0])*60.+float(timet[1])*60.+float(timet[2]))
				file_name_array.append(file_name)
				number_of_slices.append(tfile.asarray().shape[0])
				
			except:
				pass
idx = np.argsort(creation_array)
print("idx",idx)
creation_array = np.array(creation_array)[idx]
file_name_array = np.array(file_name_array)[idx]
number_of_slices = np.array(number_of_slices)[idx]
l=creation_array[0]
for i,j,k in zip(creation_array,file_name_array,number_of_slices):
	print(j,"\t",i-l,"\t",k)
	l=i	

In [ ]:
def overlap(self,bb_test,bb_gt):
        """Computes IUO between two bboxes in the form [x1,y1,x2,y2]"""
        xx1 = np.maximum(bb_test[0], bb_gt[0])
        yy1 = np.maximum(bb_test[1], bb_gt[1])
        xx2 = np.minimum(bb_test[2], bb_gt[2])
        yy2 = np.minimum(bb_test[3], bb_gt[3])
        w = np.maximum(0., xx2 - xx1)
        h = np.maximum(0., yy2 - yy1)
        wh = w * h
        area_test = (bb_test[2]-bb_test[0])*(bb_test[3]-bb_test[1])
        area_gt = (bb_gt[2]-bb_gt[0])*(bb_gt[3]-bb_gt[1])
        o = wh / (area_test + area_gt - wh)

        return o,(wh)/area_test,(wh)/area_gt

xa0 = 5
xa1 = 7
ya0 = 6
ya1 = 8

xb0 = 6
xb1 = 7
yb0 = 6
yb1 = 8
plot([xa0,xa1,xa1,xa0,xa0],[ya0,ya0,ya1,ya1,ya0],'r-')
plot([xb0,xb1,xb1,xb0,xb0],[yb0,yb0,yb1,yb1,yb0],'b-')
GT = [xa0,ya0,xa1,ya1]
test = [xb0,yb0,xb1,yb1]
print(overlap(test,GT))

In [ ]:
#Visualise avialable regions.
plot(trk_mat[0,:],trk_mat[1,:],'o')

In [ ]:
import cv2
img = cv2.imread('/Users/dwaithe/Desktop/out.tif')
x0 = -3266.7
x1 = -1866.7
y0 = 1291.7
y1 = 2291.7

figsize(12,12)
#imshow(img)
minx = np.min(trk_mat[0,:])
miny = np.min(trk_mat[1,:])
for d in out_results:
    d = d.astype(np.int32)
    #print(d)
    
    if d[0] > x0 and d[0] < x1:
        if d[1] > y0 and d[1] < y1:
            d[0] = (d[0]-minx)/0.26
            d[2] = (d[2]-minx)/0.26
            d[1] = (d[1]-miny)/0.26
            d[3] = (d[3]-miny)/0.26
            random.seed( d[4] )
            R = (random.random())  # same random number as before
            G = (random.random())  # same random number as before
            B = (random.random()) # same random number as before
            plot([d[0],d[2],d[2],d[0],d[0]],[d[1],d[1],d[3],d[3],d[1]],c=[R,G,B])
plt.axis('equal')
#plt.xlim(0,1000)
#plt.ylim(2300,3800)